# Named entity recognition

In this notebook, we will perform named entity recognition on one of the articles, and we will explain why this might be useful.

> You can use my code from the old repository for this. It might be nice to add a bit more explanation to it. Also, as this is the last Notebook, it's nice to have a bit of a general round up. You can also add a different Notebook for this of course.